In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
HOLIDAYS_FILE = 'data/holidays.csv'

In [4]:
import pandas as pd
import numpy as np
from dimdate import create_dimdate, add_holidays

dimdate = create_dimdate('2024-01-01', '2024-12-31')
dimdate

,Date,DayOfWeek,DayOfWeekText,DayOfWeekShort,IsWeekend
0,2024-01-01,1,Monday,Mon,0
1,2024-01-02,2,Tuesday,Tue,0
2,2024-01-03,3,Wednesday,Wed,0
3,2024-01-04,4,Thursday,Thu,0
4,2024-01-05,5,Friday,Fri,0
...,...,...,...,...,...
361,2024-12-27,5,Friday,Fri,0
362,2024-12-28,6,Saturday,Sat,1
363,2024-12-29,7,Sunday,Sun,1
364,2024-12-30,1,Monday,Mon,0


In [5]:
holidays = pd.read_csv(HOLIDAYS_FILE)
dimdate = add_holidays(dimdate, list(holidays['holidays']))
dimdate

,Date,DayOfWeek,DayOfWeekText,DayOfWeekShort,IsWeekend,IsHoliday,DaysToHoliday
0,2024-01-01,1,Monday,Mon,0,1,0
1,2024-01-02,2,Tuesday,Tue,0,0,1
2,2024-01-03,3,Wednesday,Wed,0,0,2
3,2024-01-04,4,Thursday,Thu,0,0,3
4,2024-01-05,5,Friday,Fri,0,0,4
...,...,...,...,...,...,...,...
361,2024-12-27,5,Friday,Fri,0,1,0
362,2024-12-28,6,Saturday,Sat,1,1,0
363,2024-12-29,7,Sunday,Sun,1,1,0
364,2024-12-30,1,Monday,Mon,0,1,0
